<a href="https://colab.research.google.com/github/saadz-khan/StyleGAN-client/blob/master/Creative_cake_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Style Transfer with tensorflow keras

## Overview

But, what is neural style transfer?

Neural style transfer is an optimization technique used to take three images, a **content** image, a **style reference** image (such as an artwork by a famous painter), and the **input** image you want to style -- and blend them together such that the input image is transformed to look like the content image, but “painted” in the style of the style image.

For example, let’s take an image of turtle and Katsushika Hokusai's *The Great Wave off Kanagawa*:

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/Green_Sea_Turtle_grazing_seagrass.jpg?raw=1" alt="Drawing" style="width: 200px;"/>
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/The_Great_Wave_off_Kanagawa.jpg?raw=1" alt="Drawing" style="width: 200px;"/>

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/wave_turtle.png?raw=1" alt="Drawing" style="width: 500px;"/>

Is this magic or just deep learning? Fortunately, this doesn’t involve any witchcraft: style transfer is a fun and interesting technique that showcases the capabilities and internal representations of neural networks.  

### We will follow the general steps to perform style transfer:

1. Visualize data
2. Basic Preprocessing/preparing our data
3. Set up loss functions 
4. Create model
5. Optimize for loss function



## Setup

### Downloading Sample Images

In [ ]:
import os
!mkdir /content/images
img_dir = '/content/images'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P {img_dir} https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg

In [ ]:
!git clone https://github.com/anishathalye/neural-style

In [ ]:
# Set up some global values here
content_path = '/content/images/Green_Sea_Turtle_grazing_seagrass.jpg' #@param[]{allow-input: true}
style_path = '/content/images/The_Great_Wave_off_Kanagawa.jpg' #@param[]{allow-input: true}
output_name = 'output' #@param[]{allow-input: true}



In [ ]:
CONTENT_WEIGHT = 5e0 #@param {type: "number"}
CONTENT_WEIGHT_BLEND = 1 #@param {type: "number"}
STYLE_WEIGHT = 5e2 #@param {type: "number"}
TV_WEIGHT = 1e2 #@param {type: "number"}
STYLE_LAYER_WEIGHT_EXP = 1 #@param {type: "number"}

LEARNING_RATE = 1e1 #@param {type: "number"}

BETA1 = 0.9 #@param {type: "number"}
BETA2 = 0.999 #@param {type: "number"}

EPSILON = 1e-08 #@param {type: "number"}

STYLE_SCALE = 1.0 #@param {type: "number"}

ITERATIONS = 1000 #@param {type: "number"}

### Visualize the Inputs

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img


def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)


plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

### Run the Scripts

In [ ]:
!python run neural_style.py --content {content} --styles {style} --output {output} --iteration {ITERATIONS} --print-iteration --progress-plot\
/ --content-weight-blend {CONTENT_WEIGHT_BLEND} --content-weight {CONTENT_WEIGHT} --style-weight {STYLE_WEIGHT} --style-layer-weight-exp {STYLE_LAYER_WEIGHT_EXP}\
/ --tv-weight {TV_WEIGHT} --learning-rate {LEARNING_RATE} --beta1 {BETA1} --beta2 {BETA2} --eps {EPSILON}